# PRÀCTICA: VISUALITZACIÓ DE DADES
*Arnau Sanz*


### Les zones més accidentades de la ciutat de Barcelona
#### Com de segur es ser un/a vianant?

El primer pas és instal·lar i importar totes les llibreries que s'empraran al llarg de tota la pràctica.

In [1]:
import importlib

libraries = {
    'pandas': 'pd',
    'numpy': 'np',
    'folium': 'folium',
    'geopy': 'geopy',
    'geopandas': 'gpd',
    'shapely': 'shapely',
    'matplotlib': 'plt',
    'plotly': 'px',
    'dash': 'dash',
}

missing_libraries = []

# If any library is missing, it will be installed at this point. It may take some minutes depending on the number of libraris it will have to install.
for library, alias in libraries.items():
    try:
        importlib.import_module(library)
    except ImportError:
        missing_libraries.append((library, alias))

if missing_libraries:
    for library, alias in missing_libraries:
        !pip install {library}
else:
    print("All libraries installed")

All libraries installed


In [2]:
# Imports for entire project
import os
import pandas as pd
import numpy as np
import folium
import json
from geopy.geocoders import Nominatim
import geopandas as gpd
import shapely.wkt as wkt
import matplotlib.pyplot as plt
from folium.plugins import FastMarkerCluster
import plotly.express as px
import dash
from dash import html, dcc
from shapely.geometry import mapping

Llegim els documents. Com que les nostres dades estan repartides en diferents documents, els llegirem tots de manera individual i després els ajuntarem, assegurant-nos que les columnes són les mateixes i estan en el mateix ordre. 

El primer que s'ha de fer, però, és comprovar l'extensió de tots els documents.

In [3]:
# Read documents from directory "raw data", previously downloaded.
dir = "raw_data"
current_dir = os.getcwd()
os.chdir(dir)
file_type=set()
for file in os.listdir():
    file_type.add(file[-3:])
a=str(file_type)

Podem comprovar com tenim més d'una extensió, per tant, la lecutra dels documents s'haurà de fer tenint aquest fet en compte. Concretament les extensions dels documents són: **csv** i **xml**. En aquest cas, doncs, recorrem els documents i els llegim, guardant la informació de cadascun en un DataFrame individual, per la posterior comprovació de coherència entre els diferents subconjunts de dades.

In [4]:
# Show all the documents with our raw data
dataset_raw = list()
for file in os.listdir():
    if(file[-3:] == 'csv'):
        # print file name
        print(file)
        # encoding = latin1, not working with utf-8 or utf-16
        read_df = pd.read_csv(file, encoding='latin1')
        dataset_raw.append(read_df)
    elif(file[-3:] == 'xml'):
        # print file name
        print(file)
        read_df = pd.read_xml(file, encoding='latin1')
        dataset_raw.append(read_df)
os.chdir(current_dir)

2022_accidents_gu_bcn.csv
2011_ACCIDENTS_GU_BCN_2011.csv
2015_ACCIDENTS_GU_BCN_2015.xml
2013_ACCIDENTS_GU_BCN_2013.csv
2014_ACCIDENTS_GU_BCN_2014.csv
2012_ACCIDENTS_GU_BCN_2012.csv
2020_accidents_gu_bcn.csv
2018_accidents_gu_bcn.csv
2017_accidents_gu_bcn.csv
2010_ACCIDENTS_GU_BCN_2010.csv
2016_accidents_gu_bcn.csv
2019_accidents_gu_bcn.csv
2021_accidents_gu_bcn.csv


In [5]:
# Show firts 5 rows from the first data source to ensure it worked
dataset_raw[0].head()

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal_caption,Descripcio_dia_setmana,NK_Any,...,Descripcio_causa_vianant,Numero_morts,Numero_lesionats_lleus,Numero_lesionats_greus,Numero_victimes,Numero_vehicles_implicats,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,2022S007749,-1,Desconegut,-1,Desconegut,-1,Fernando Pessoa ...,17,Dilluns,2022,...,No Ã©s causa del vianant,0,1,0,1,2,432684.59,4588039.29,2.193098,41.439261
1,2022S003422,-1,Desconegut,-1,Desconegut,-1,Bac de Roda / Ramon TurrÃ³ ...,Desconegut,Dissabte,2022,...,No Ã©s causa del vianant,0,2,0,2,2,433936.91,4584121.59,2.208515,41.404080
2,2022S003346,-1,Desconegut,-1,Desconegut,-1,Viladrosa ...,97-103,Dimecres,2022,...,No Ã©s causa del vianant,0,1,0,1,2,431308.23,4588533.51,2.176568,41.443596
3,2022S006073,-1,Desconegut,-1,Desconegut,-1,A Zona Franca / NÃºmero 3 Zona Franca ...,Desconegut,Dissabte,2022,...,No Ã©s causa del vianant,0,0,0,0,1,428020.43,4576283.26,2.138671,41.332974
4,2022S005123,-1,Desconegut,-1,Desconegut,-1,Carles Pi i Sunyer ...,10,Divendres,2022,...,No Ã©s causa del vianant,0,1,0,1,2,431029.88,4582053.59,2.173975,41.385209


Un cop hem carregat les dades, comprovem que tinguin les mateixes columnes, amb el mateix tipus de dades i els mateixos noms. Com que les descàrregues s'han fet de la matexia font, és d'esperar que sigui així, però ho comprovem per evitar futures problemàtiques.

In [6]:
# First Dataset Columns
column_names_1 = dataset_raw[0].columns.to_list()
column_types_1 = dataset_raw[0].dtypes.to_list()
for dataset in dataset_raw:
    print(dataset.columns.to_list()==column_names_1, dataset.dtypes.to_list()==column_types_1)

True True
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False False
False True


Podem comprovar com no totes les fonts de dades contenen la mateixa informació o, com a mínim, ordenada de la mateixa manera. Haurem de tractar asquest aspecte per unificar la base de dades amb la que treballarem i evitar errors o problemàtiques futures.
El primer pas és comprovar si tots els datasets tenen el mateix nombre de columnes.

In [7]:
for d in dataset_raw:
    print(len(d.columns))

25
25
24
25
25
25
27
27
27
25
27
27
25


Es pot observar que no tots els datasets tenen el mateix nombre de columnes. Així doncs, el que farem serà comprobar quines són les columnes comunes i quines no. Posteriorment, la idea és generar un nou dataset que contingui totes les columnes (amb el mateix ordre) i, a partir d'aquí, en funció de quina sigui la informació que sigui comuna i la que no, prendre una decisió o una altra. Pot ser que es puguin obviar les columnes no comunes o, en cas que la informació que aportin sigui molt valuosa, optar per tractar-ho d'alguna altra forma.

In [8]:
df_columns = pd.DataFrame()
for i in range(len(dataset_raw)):
    # As max number of columns = 27, we set all of them to len 27
    list_aux = dataset_raw[i].columns.to_list()
    while len(list_aux) < 27:
        list_aux.append(np.nan)
    df_columns[i]=list_aux

A la taula que tenim a continuació, podem veure les columnes i les seves diferències. Realment, les diferències principals no són molt destacables, ja que fan referència a dades de les quals ja en tenim la mateixa informació en altres columnes. Així doncs, unificarem tots els datasets, els combinarem en un de sol, i decidirem què fer-ne d'aquestes columnes que continguin buits.

In [9]:
df_columns

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Numero_expedient,Número d'expedient,Numerodexpedient,Número d'expedient,Número d'expedient,Número d'expedient,Numero_expedient,Numero_expedient,Numero_expedient,Número d'expedient,Numero_expedient,Numero_expedient,Numero_expedient
1,Codi_districte,Codi districte,Codidistricte,Codi districte,Codi districte,Codi districte,Codi_districte,Codi_districte,Codi_districte,Codi districte,Codi_districte,Codi_districte,Codi_districte
2,Nom_districte,Nom districte,Nomdistricte,Nom districte,Nom districte,Nom districte,Nom_districte,Nom_districte,Nom_districte,Nom districte,Nom_districte,Nom_districte,Nom_districte
3,Codi_barri,NK barri,NKbarri,NK barri,NK barri,NK barri,Codi_barri,Codi_barri,Codi_barri,NK barri,Codi_barri,Codi_barri,Codi_barri
4,Nom_barri,Nom barri,Nombarri,Nom barri,Nom barri,Nom barri,Nom_barri,Nom_barri,Nom_barri,Nom barri,Nom_barri,Nom_barri,Nom_barri
5,Codi_carrer,Codi carrer,Codicarrer,Codi carrer,Codi carrer,Codi carrer,Codi_carrer,Codi_carrer,Codi_carrer,Codi carrer,Codi_carrer,Codi_carrer,Codi_carrer
6,Nom_carrer,Nom carrer,Nomcarrer,Nom carrer,Nom carrer,Nom carrer,Nom_carrer,Nom_carrer,Nom_carrer,Nom carrer,Nom_carrer,Nom_carrer,Nom_carrer
7,Num_postal_caption,Num postal caption,Numpostalcaption,Num postal caption,Num postal caption,Num postal caption,Num_postal_caption,Num_postal,Num_postal,Num postal caption,Num_postal,Num_postal_caption,Num_postal_caption
8,Descripcio_dia_setmana,Descripció dia setmana,Descripciodiasetmana,Descripció dia setmana,Descripció dia setmana,Descripció dia setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripció dia setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripcio_dia_setmana
9,NK_Any,Dia de setmana,Diadesetmana,Dia de setmana,Dia de setmana,Dia de setmana,Dia_setmana,Dia_setmana,Dia_setmana,Dia de setmana,Dia_setmana,Dia_setmana,NK_Any


Per a poder unificar les columnes, mirarem cada fila del DataFrame auxiliar que ens hem creat anteriorment i en cada cas, unificarem el nom de la columna (doncs també ens trobem en columnes que aporten la mateixa informació, però amb noms diferents) i, en cas que la columna no existeixi en algun dataset en concret, la deixarem buida.

Per determinar el nom de les columnes, agafarem un dels datasets més complerts i que tingui uns noms de columnes el màxim d'estàndard possible. En aquest cas, agafarem el dataset número 7, ja que els noms no contenen espais, accents ni caràcters especials.

In [10]:
# Final columns
final_columns = df_columns[6].to_list()
print(final_columns)

['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri', 'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal_caption', 'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'NK_Any', 'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn', 'Descripcio_causa_vianant', 'Numero_morts', 'Numero_lesionats_lleus', 'Numero_lesionats_greus', 'Numero_victimes', 'Numero_vehicles_implicats', 'Coordenada_UTM_X', 'Coordenada_UTM_Y', 'Longitud', 'Latitud']


In [11]:
df_columns.loc[9:24,0].values

array(['NK_Any', 'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia',
       'Descripcio_torn', 'Descripcio_causa_vianant', 'Numero_morts',
       'Numero_lesionats_lleus', 'Numero_lesionats_greus',
       'Numero_victimes', 'Numero_vehicles_implicats',
       'Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50', 'Longitud',
       'Latitud'], dtype=object)

En aquest punt, el que fem és "moure" de lloc les columnes de tal manera que en tots els diferents datasets, es trobin en la mateixa posició ja qu el'objectiu serà combinar-los tots.

In [12]:
# Dataset 0
df_columns.loc[11:26,0] = df_columns.loc[9:24,0].values
df_columns.loc[9:10, 0] = np.nan
# Dataset 3
df_columns.loc[24,2] = df_columns.loc[23,2]
df_columns.loc[23,2] = df_columns.loc[22,2]
df_columns.loc[22,2] = df_columns.loc[21,2]
df_columns.loc[21,2] = df_columns.loc[18,2]
df_columns.loc[18,2] = np.nan
# Dataset 13
df_columns.loc[11:26,12] = df_columns.loc[9:24,12].values
df_columns.loc[9:10, 12] = np.nan
# Replace all NaN by an aux name, to set the new columns
for i in range(len(df_columns.loc[:, 0])):
    df_columns.loc[i, :] = df_columns.loc[i, :].replace(np.nan, "AUX"+str(i))

In [13]:
df_columns

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Numero_expedient,Número d'expedient,Numerodexpedient,Número d'expedient,Número d'expedient,Número d'expedient,Numero_expedient,Numero_expedient,Numero_expedient,Número d'expedient,Numero_expedient,Numero_expedient,Numero_expedient
1,Codi_districte,Codi districte,Codidistricte,Codi districte,Codi districte,Codi districte,Codi_districte,Codi_districte,Codi_districte,Codi districte,Codi_districte,Codi_districte,Codi_districte
2,Nom_districte,Nom districte,Nomdistricte,Nom districte,Nom districte,Nom districte,Nom_districte,Nom_districte,Nom_districte,Nom districte,Nom_districte,Nom_districte,Nom_districte
3,Codi_barri,NK barri,NKbarri,NK barri,NK barri,NK barri,Codi_barri,Codi_barri,Codi_barri,NK barri,Codi_barri,Codi_barri,Codi_barri
4,Nom_barri,Nom barri,Nombarri,Nom barri,Nom barri,Nom barri,Nom_barri,Nom_barri,Nom_barri,Nom barri,Nom_barri,Nom_barri,Nom_barri
5,Codi_carrer,Codi carrer,Codicarrer,Codi carrer,Codi carrer,Codi carrer,Codi_carrer,Codi_carrer,Codi_carrer,Codi carrer,Codi_carrer,Codi_carrer,Codi_carrer
6,Nom_carrer,Nom carrer,Nomcarrer,Nom carrer,Nom carrer,Nom carrer,Nom_carrer,Nom_carrer,Nom_carrer,Nom carrer,Nom_carrer,Nom_carrer,Nom_carrer
7,Num_postal_caption,Num postal caption,Numpostalcaption,Num postal caption,Num postal caption,Num postal caption,Num_postal_caption,Num_postal,Num_postal,Num postal caption,Num_postal,Num_postal_caption,Num_postal_caption
8,Descripcio_dia_setmana,Descripció dia setmana,Descripciodiasetmana,Descripció dia setmana,Descripció dia setmana,Descripció dia setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripció dia setmana,Descripcio_dia_setmana,Descripcio_dia_setmana,Descripcio_dia_setmana
9,AUX9,Dia de setmana,Diadesetmana,Dia de setmana,Dia de setmana,Dia de setmana,Dia_setmana,Dia_setmana,Dia_setmana,Dia de setmana,Dia_setmana,Dia_setmana,AUX9


A la cel·la anterior, es pot veure com s'han unificat les columnes dels diferents datasets perquè estiguin en el mateix ordre i, per tant, es pugui unificar tot el conjunt de dades en una sola taula i treballar-hi de manera més còmode, ràpida, i eficient. En aquest punt, doncs, hem de reordenar els datasets individuals i combinar-los.

In [14]:
# Create a list with the new datasets all of them with the same columns.
dataset_unif = list()
for idx_d in range(len(dataset_raw)):
    aux_unif_dataset = pd.DataFrame(columns = df_columns[idx_d].to_list())
    for c in df_columns[idx_d].to_list():
        if c in (dataset_raw[idx_d].columns.to_list()):
            aux_unif_dataset[c] = dataset_raw[idx_d][c]
        else:
            aux_unif_dataset[c] = np.nan
    aux_unif_dataset.columns = final_columns
    dataset_unif.append(aux_unif_dataset)

In [15]:
dataset_unif[0].head()

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal_caption,Descripcio_dia_setmana,Dia_setmana,...,Descripcio_causa_vianant,Numero_morts,Numero_lesionats_lleus,Numero_lesionats_greus,Numero_victimes,Numero_vehicles_implicats,Coordenada_UTM_X,Coordenada_UTM_Y,Longitud,Latitud
0,2022S007749,-1,Desconegut,-1,Desconegut,-1,Fernando Pessoa ...,17,Dilluns,NaN,...,No Ã©s causa del vianant,0,1,0,1,2,432684.59,4588039.29,2.193098,41.439261
1,2022S003422,-1,Desconegut,-1,Desconegut,-1,Bac de Roda / Ramon TurrÃ³ ...,Desconegut,Dissabte,NaN,...,No Ã©s causa del vianant,0,2,0,2,2,433936.91,4584121.59,2.208515,41.404080
2,2022S003346,-1,Desconegut,-1,Desconegut,-1,Viladrosa ...,97-103,Dimecres,NaN,...,No Ã©s causa del vianant,0,1,0,1,2,431308.23,4588533.51,2.176568,41.443596
3,2022S006073,-1,Desconegut,-1,Desconegut,-1,A Zona Franca / NÃºmero 3 Zona Franca ...,Desconegut,Dissabte,NaN,...,No Ã©s causa del vianant,0,0,0,0,1,428020.43,4576283.26,2.138671,41.332974
4,2022S005123,-1,Desconegut,-1,Desconegut,-1,Carles Pi i Sunyer ...,10,Divendres,NaN,...,No Ã©s causa del vianant,0,1,0,1,2,431029.88,4582053.59,2.173975,41.385209


Podem veure com efectivament hem aconseguit posar els diferents datasets en un mateix format i, per tant, ara els podem combinar per generar un dataset únic que contingui finalment, totes les dades juntes i poder començar a treballar amb aquest dataset.

In [16]:
# Generate the final DataFrame
df = pd.concat(dataset_unif)
df.reset_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118653 entries, 0 to 118652
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   index                      118653 non-null  int64  
 1   Numero_expedient           118653 non-null  object 
 2   Codi_districte             118653 non-null  int64  
 3   Nom_districte              118653 non-null  object 
 4   Codi_barri                 118653 non-null  object 
 5   Nom_barri                  118653 non-null  object 
 6   Codi_carrer                118653 non-null  int64  
 7   Nom_carrer                 118651 non-null  object 
 8   Num_postal_caption         113850 non-null  object 
 9   Descripcio_dia_setmana     118653 non-null  object 
 10  Dia_setmana                102995 non-null  object 
 11  Descripcio_tipus_dia       102995 non-null  object 
 12  NK_Any                     118653 non-null  int64  
 13  Mes_any                    11

### Les zones més accidentades de la ciutat

##### Segmentació per barris

En aquest punt, es comença l'anàlisi i visualització del nombre d'accidents en funció de la zona de la ciutat. *En aquest cas, es mostrarà segmentat per **barris***.

El primer que caldrà fer, serà tenir la informació unificada de tal manera que no tinguem noms de barris duplicats, noms erronis, etc.

In [17]:
print(sorted(df["Nom_barri"].unique()))

['BarÃ³ de Viver', 'Baró de Viver', 'Can BarÃ³', 'Can Baró', 'Can Peguera', 'Canyelles', 'Ciutat Meridiana', 'Desconegut', 'Diagonal Mar i el Front MarÃ\xadtim del Poblenou', 'Diagonal Mar i el Front Marítim del Poblenou', 'Horta', 'Hostafrancs', 'Montbau', 'Navas', 'Pedralbes', 'Porta', 'ProvenÃ§als del Poblenou', 'Provençals del Poblenou', 'Sant Andreu', 'Sant Antoni', 'Sant GenÃ\xads dels Agudells', 'Sant Genís dels Agudells', 'Sant Gervasi - Galvany', 'Sant Gervasi - la Bonanova', 'Sant MartÃ\xad de ProvenÃ§als', 'Sant Martí de Provençals', 'Sant Pere, Santa Caterina i la Ribera', 'Sants', 'Sants - Badal', 'SarriÃ', 'SarriÃ\xa0', 'Sarrià', 'Torre BarÃ³', 'Torre Baró', 'Vallbona', 'Vallcarca i els Penitents', 'Vallvidrera, el Tibidabo i les Planes', 'Verdun', 'Vilapicina i la Torre Llobeta', 'el Baix GuinardÃ³', 'el Baix Guinardó', 'el Barri GÃ²tic', 'el Barri Gòtic', 'el BesÃ²s i el Maresme', 'el Besòs i el Maresme', 'el Bon Pastor', "el Camp d'en Grassot i GrÃ\xa0cia Nova", "el Ca

Podem veure com hi ha una gran quantitat de noms de barris mal escrits. A l'hora d'importar hem utilitzat una codificació concreta ***latin1***, tot i així, veiem que hi segueix havent problemes en aquest sentit, potser degut a una mala descàrrega de les dades. Ho hem de tractar.

In [18]:
# Unifiquem el nom del barri
sorted(df["Nom_barri"].unique())
df.replace({
    "Ã§" : "ç",
    "Ã\xad" : "í",
    "Ã\xa0" : "à",
    "Ã³" : "ó",
    "Ã²" : "ò", 
    "Ã©" : "é",
    "Ã" : "à",
},
    inplace=True, regex=True)

In [19]:
print(sorted(df["Nom_barri"].unique()))

['Baró de Viver', 'Can Baró', 'Can Peguera', 'Canyelles', 'Ciutat Meridiana', 'Desconegut', 'Diagonal Mar i el Front Marítim del Poblenou', 'Horta', 'Hostafrancs', 'Montbau', 'Navas', 'Pedralbes', 'Porta', 'Provençals del Poblenou', 'Sant Andreu', 'Sant Antoni', 'Sant Genís dels Agudells', 'Sant Gervasi - Galvany', 'Sant Gervasi - la Bonanova', 'Sant Martí de Provençals', 'Sant Pere, Santa Caterina i la Ribera', 'Sants', 'Sants - Badal', 'Sarrià', 'Torre Baró', 'Vallbona', 'Vallcarca i els Penitents', 'Vallvidrera, el Tibidabo i les Planes', 'Verdun', 'Vilapicina i la Torre Llobeta', 'el Baix Guinardó', 'el Barri Gòtic', 'el Besòs i el Maresme', 'el Bon Pastor', "el Camp d'en Grassot i Gràcia Nova", "el Camp de l'Arpa del Clot", 'el Carmel', 'el Clot', 'el Coll', 'el Congrés i els Indians', 'el Fort Pienc', 'el Guinardó', 'el Parc i la Llacuna del Poblenou', 'el Poble Sec', 'el Poble-sec', 'el Poblenou', 'el Putxet i el Farró', 'el Raval', 'el Turó de la Peira', "l'Antiga Esquerra de l

In [20]:
barris_group = df.groupby("Nom_barri").count()
barris_group

,index,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Codi_carrer,Nom_carrer,Num_postal_caption,Descripcio_dia_setmana,Dia_setmana,...,Descripcio_causa_vianant,Numero_morts,Numero_lesionats_lleus,Numero_lesionats_greus,Numero_victimes,Numero_vehicles_implicats,Coordenada_UTM_X,Coordenada_UTM_Y,Longitud,Latitud
Nom_barri,,,,,,,,,,,,,,,,,,,,,
Baró de Viver,277,277,277,277,277,277,277,271,277,232,...,277,250,277,277,277,277,277,277,159,159
Can Baró,239,239,239,239,239,239,239,234,239,209,...,239,218,239,239,239,239,239,239,104,104
Can Peguera,82,82,82,82,82,82,82,80,82,71,...,82,74,82,82,82,82,82,82,39,39
Canyelles,472,472,472,472,472,472,472,463,472,402,...,472,430,472,472,472,472,472,472,250,250
Ciutat Meridiana,156,156,156,156,156,156,156,153,156,135,...,156,148,156,156,156,156,156,156,73,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
la Vila Olímpica del Poblenou,1399,1399,1399,1399,1399,1399,1399,1347,1399,1218,...,1399,1295,1399,1399,1399,1399,1399,1399,728,728
la Vila de Gràcia,1985,1985,1985,1985,1985,1985,1985,1912,1985,1728,...,1985,1808,1985,1985,1985,1985,1985,1985,957,957
les Corts,3591,3591,3591,3591,3591,3591,3591,3419,3591,3123,...,3591,3267,3591,3591,3591,3591,3591,3591,1890,1890


Un cop hem unificat i estandaritzat el nom dels diferents barris de la ciutat, podem crear un mapa interactiu en el que podrem visualitzar la distribució dels accidents en funció de la zona (*del barri*). Emprerem la llibreria **folium**. 

In [21]:
# Define the map and set controls as disabled
barcelona_map = folium.Map(location = [41.3851, 2.1734],
                           zoom_start = 12,
                           zoom_control = False,
                           scrollWheelZoom = False,
                           doubleClickZoom = False,
                           dragging = False,
                           name="Map View 1"
                          )
# Add a second Layer Style
folium.TileLayer('cartodbpositron', name='cartodbpositron').add_to(barcelona_map)

Per a poder veure amb claredat i ràpidament la distribució del nombre d'accidents per barris, el que farem serà descarregar-nos, directament des del web de l'Ajuntament de Barcelona, un fitxer GeoJSON amb les coordenades que delimiten el diferents barris i, d'aquesta manera, poder gràficar els límits de cadscun d'ells.

In [22]:
# Download GeoJSON file and save it in the root folder
url_geojson_regions = "https://opendata-ajuntament.barcelona.cat/resources/bcn/EstadisticaUnitatsAdministratives/BarcelonaCiutat_Barris.csv"
regions_geojson = "regions.geojson"
gpd.read_file(url_geojson_regions).to_file(regions_geojson, driver='GeoJSON')

A continuació, el que farem serà agrupar el nombre d'accidents per barri de tal manera que es pugui fer el gràfic desitjat. Descartarem els registres dels quals no en tenim informació.

In [23]:
num_crashes_df = df[df["Nom_barri"]!="Desconegut"].groupby(["Nom_barri"]).count()["Numero_expedient"]
num_crashes_df.head(5)

Nom_barri
Baró de Viver       277
Can Baró            239
Can Peguera          82
Canyelles           472
Ciutat Meridiana    156
Name: Numero_expedient, dtype: int64

In [24]:
# Define the DataFrame with the information that will be shown in the map
geojson_data = regions_geojson
df_zone = pd.DataFrame()
zone_name = list()
zone_coord = list()
num_crashes = list()

with open(geojson_data) as file:
    data = json.load(file)
    for feature in data['features']:
        # Get properties for each zone in the GeoJSON file
        zone_name.append(feature['properties']['nom_barri'])
        wkt_polygon = feature['properties']['geometria_wgs84']
        zone_coord.append(wkt.loads(wkt_polygon))
        num_crashes.append(num_crashes_df[feature['properties']['nom_barri']])
df_zone["Name"] = zone_name
df_zone["Coordinates"] = zone_coord
df_zone["Number of Crashes"] = num_crashes

Com que l'arxiu GeoJSOn que ens hem descarregat conté les coordenades de dues maneres diferents, optarem per crear un nou GeoJSON temporal, que usarem al llarg de la pràctica, però que no guardarem, per a poder crear els gràfics de manera còmode.

In [25]:
# Creation of the new GeoJSON
features = []

for _, row in df_zone.iterrows():
    # Convert Polygon to string
    geometry = mapping(row["Coordinates"])

    # Create a new GeoJSON with only the requiered information
    feature = {
        "type": "Feature",
        "properties": {
            "Name": row["Name"],
            "Number of Crashes": row["Number of Crashes"]
        },
        "geometry": geometry
    }

    # Set the features requiered for each zone
    features.append(feature)

geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

Un cop tenim la informació que volem sobre les coordenades i el nom de cada barri, el que fem és crear una capa interactiva amb Choropleth.

In [26]:
# Create interactive layer over the zones map
folium.Choropleth(
    geo_data=geojson_data,
    data=df_zone,
    columns=['Name', 'Number of Crashes'],
    key_on='feature.properties.Name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Crashes',
    control=False
).add_to(barcelona_map)

# Define a style function for the polygons
def style_function(feature):
    return {
        'fillColor': 'YlOrRd',
        'fillOpacity': 0.2,
        'color': 'black',
        'weight': 1
    }

# Add the GeoJSON layer to the map with the style function and add a highlight function for when the mouse passes over each zone
folium.GeoJson(
    geojson_data,
    name='Zones Crashes Density',
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Name', 'Number of Crashes'],
        aliases=['Name', 'Number of Crashes'],
        localize=True
    ),
    highlight_function=lambda x: {'weight': 2, "fillOpacity": 0}
).add_to(barcelona_map)

# Add a layer control to the map
folium.LayerControl().add_to(barcelona_map)

Abans de poder creuar la informació que tenim sobre el nombre d'accidents per barri, amb les dades de les coordenades de cada barri i poder dibuixar el mapa, ens hem d'assegurar que el nom dels barris és igual a les dues bandes. En cas que no, no es mostrarà la informació correctament al mapa.

In [27]:
# Compare df_zone "zone_name" with df["Nom_barri"]
set(df["Nom_barri"]).difference(set(df_zone["Name"]))

{'Desconegut', 'el Poble Sec'}

Podem veure com tenim una diferència (a part dels barris desconeguts, que els hem ignorat). La solucionem i podem continuar.

In [28]:
df.replace({"el Poble Sec" : "el Poble-sec"}, inplace=True)

### Dibuixem el gràfic interactiu
#### Característiques:

- No es permet el moviment ni el zoom de cap manera. La idea d'aquest gràfic és mostrar la informació de manera genèrica i ens interessa que es vegi per complet tota l'estona.
- Tenim un control que ens permet mostrar o ocultar una de les capes, a més de permetre canviar el tipus de mapa que volem visualitzar:
    - Capa 1: Els barris, dibuixats d'un color en funció del nombre d'accidents (que es defineix a la llegenda). ***No es pot controlar interactivament***.
    - Capa 2: Capa interactiva que ens mostra el nom i el nombre d'accidents de cada barri al passer el ratolí. ***Sí que es pot controlar interactivament***.

In [ ]:
barcelona_map

Podem veure com hi ha un barri que destaca més que la resta en quant a nombre d'accidents. Aquest barri és el de la Dreta de l'Eixample, represetant amb color vermell. En aquest barri trobem els carrers més transitats de la ciutat així com les zones més cèntriques i turístiques, de manera que l'afluència de cotxes és alta, però la de viantants també. Ens centrem ara, doncs, a estudiar a quines "subzones" hi ha més accidents.

In [30]:
# Get only the rows of the specified zone that have coordinates data
df[df["Nom_barri"] == "la Dreta de l'Eixample"][["Latitud", "Longitud"]].info()

<class 'pandas.core.frame.DataFrame'>
Index: 13176 entries, 253 to 118116
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Latitud   6812 non-null   float64
 1   Longitud  6812 non-null   float64
dtypes: float64(2)
memory usage: 308.8 KB


### Dibuixem el gràfic interactiu
#### Característiques:

- Sí es permet el moviment ni el zoom de cap manera. La idea d'aquest gràfic és mostrar la informació de manera precisa i ens interessa que pugui fer zoom per veure les subzones i carrers amb més detall.
- Tenim un control que ens permet mostrar o ocultar les dues capes:
    - Capa 1: La delimitació del barri en qüestió.
    - Capa 2: El nombre d'accidents registrats en una subzona en concret. A mesura que es fa zooom al mapa, s'autoadapta.

In [31]:
# Coordinates of Dreta de l'Eixample
zone_lat, zone_lon = 41.394, 2.167

# Create a new map centered on this zone and with more zoom. In this case, it will have controls to move and zoom.
de_map = folium.Map(location=[zone_lat, zone_lon],
                        zoom_start=15)

# Get coordinates of the limits of the zone, from our previous geojson_data
coordinates = None
for feature in geojson_data['features']:
    if feature['properties']['Name'] == "la Dreta de l'Eixample":
        coordinates = feature['geometry']['coordinates']

if coordinates:
    # Create the GeoJson Polygon and add it to the new map
    folium.GeoJson(
        name = "Zone limits",
        data={
            'type': 'Feature',
            'geometry': {
                'type': 'Polygon',
                'coordinates': coordinates
            },
            'properties': {}
        },
        style_function=lambda x: {'fillColor': 'transparent', 'color': 'black'},
    ).add_to(de_map)
    
# We ignore the rows that doesn't have coordinates information, as we won't be able to draw it into the new map
data = df[(df["Nom_barri"] == "la Dreta de l'Eixample") & (~df["Latitud"].isna())][["Latitud", "Longitud"]].copy()


# Crear el objeto FastMarkerCluster con la función icon_create_function personalizada
FastMarkerCluster(data, name="Number of crashes").add_to(de_map)


# Mostrar el mapa
folium.LayerControl().add_to(de_map)

In [32]:
de_map

Aquí podem observar com les zones mé concorregudes habitualment per vianants o zones que tenen un alt volumn de trànsit, són aquelles on es concentren més accidents. Això és evident. Tanmateix, els carrers amb més trànsit de vehicles també passen per altres barris on el volum d'accidents no era tan notori, per tant, seria interessant veure la implicació dels vianants en aquesta zona en concret.

In [33]:
dreta_eixample_vianant = df.loc[data.index][["Numero_expedient", "Descripcio_causa_vianant"]].groupby(["Descripcio_causa_vianant"]).count()
index_no_data=set(df.index.values).difference(set(data.index.values))

In [34]:
no_dreta_eixample_vianant = df.loc[list(index_no_data)][["Numero_expedient", "Descripcio_causa_vianant"]].groupby(["Descripcio_causa_vianant"]).count()


In [35]:
dreta_eixample_vianant

,Numero_expedient
Descripcio_causa_vianant,
Altres,80
Creuar per fora pas de vianants,139
Desconegut,598
Desobeir altres senyals,1
Desobeir el senyal del semàfor,230
No és causa del vianant,5733
Transitar a peu per la calçada,31


In [36]:
no_dreta_eixample_vianant

,Numero_expedient
Descripcio_causa_vianant,
Altres,1236
Creuar per fora pas de vianants,2514
Desconegut,48864
Desobeir altres senyals,36
Desobeir el senyal del semàfor,2425
No és causa del vianant,56269
Transitar a peu per la calçada,497


In [37]:
df_pedestrian = df[["Numero_expedient", "Nom_barri", "Descripcio_causa_vianant"]].copy()
def pedestrian_fault_correction(x):
    if(x=="No és causa del  vianant"):
        return "No"
    else:
        return "Yes"
def zone_name_correction(x):
    if(x=="la Dreta de l'Eixample"):
        return "la Dreta de l'Eixample"
    else:
        return "Other Zones"
df_pedestrian_corrected = df_pedestrian[(df["Descripcio_causa_vianant"]!="Desconegut") & (df["Descripcio_causa_vianant"]!="Altres")].copy()
df_pedestrian_corrected["Descripcio_causa_vianant"] = df_pedestrian_corrected["Descripcio_causa_vianant"].apply(pedestrian_fault_correction)
df_pedestrian_corrected["Nom_barri"] = df_pedestrian_corrected["Nom_barri"].apply(zone_name_correction)

In [38]:
df_pedestrian_corrected.head()

,Numero_expedient,Nom_barri,Descripcio_causa_vianant
0,2022S007749,Other Zones,No
1,2022S003422,Other Zones,No
2,2022S003346,Other Zones,No
3,2022S006073,Other Zones,No
4,2022S005123,Other Zones,No


Per poder observar is hi ha alguna relació entre el nombre de vianants i la proporció en que aquests són la causa principal de l'accident, farem un histograma interactiu que ens permeti comparar aquest barri en concret amb la resta de barris en general (agrupats). ***Per a fer-ho, emprarem la llibreria Dash i Plotly.***

### Dibuixem el gràfic interactiu
#### Característiques:

- Hi ha un selector que permet escollir de quina zona es vol graficar l'histograma:
    - Zona 1: El barri de la Dreta de l'Eixample.
    - Zona 2: La resta de barris agrupats.
- Al passar el ratolí per sobre del gràfic, dóna informació numèrica precisa.

In [39]:
# Create Dash application
app = dash.Dash(__name__)


# Define the 2 options of the dropdown
dropdown_options = [
    {"label": "la Dreta de l'Eixample", "value": "la Dreta de l'Eixample"},
    {"label": "Altres Barris", "value": "Altres Barris"}
]


app.layout = html.Div([
    # Set dropdown options
    dcc.Dropdown(
        id="dropdown",
        options=dropdown_options,
        value=None,
        placeholder="Barri",
    ),
    dcc.Graph(id="graph"),
])

# Update the graph when changing the dropdown choice
@app.callback(
    dash.dependencies.Output("graph", "figure"),
    [dash.dependencies.Input("dropdown", "value")]
)
def update_graph(selected_option):
    filtered_data = df_pedestrian_corrected[df_pedestrian_corrected["Nom_barri"] == selected_option]
    
    # Create the interactive histogram (by percent)
    fig = px.histogram(filtered_data, x="Descripcio_causa_vianant", histnorm="percent")

    return fig

# Run Dash application
if __name__ == "__main__":
    app.run_server(debug=True)

S'observa que hi ha una diferència d'un 2% entre la resta de barris i el de la Dreta de l'Eixample, cosa que és una diferència significativa (en total més de 150 casos). Així doncs, podem arribar a la conclusió que el fet de tenir més trànsit de vianants afecta inclús a la proporció en què aquests són els causants dels accidents i, per tant, per part de l'ajuntament, s'hauria de prendre alguna mesura. No obstant això, podríem mirar d'esbrinar millor quines són les causes concretes d'aquests accidents.

In [40]:
# Get rows where the pedestrian fault is true
df_pedestrian_fault = df.loc[df_pedestrian_corrected[df_pedestrian_corrected["Descripcio_causa_vianant"]=="Yes"].index][["Numero_expedient", "Descripcio_causa_vianant", "Numero_morts"]].groupby("Descripcio_causa_vianant").count()

In [41]:
# Show the causes of the pedestrain fault crashes
display(df_pedestrian_fault)

,Numero_expedient,Numero_morts
Descripcio_causa_vianant,,
Creuar per fora pas de vianants,2653,2444
Desobeir altres senyals,37,36
Desobeir el senyal del semàfor,2655,2431
Transitar a peu per la calçada,528,491
